# RVC v2 Disconnected

**Notebook hastily written by [Kit Lemonfoot](https://huggingface.co/Kit-Lemonfoot) / [Noel Shirogane's High Flying Birds](https://www.youtube.com/@NoelShiroganesHighFlyingBirds)**

*Based on the work of the [RVC Project](https://github.com/RVC-Project), [Mangio261](https://github.com/Mangio621/), [Kalomaze](https://github.com/kalomaze), [Alexlnkp](https://github.com/alexlnkp), the [Pony Preservation Project](https://boards.4channel.org/mlp/catalog#s=Pony%20Preservation%20Project), and many others in the RVC / voice AI community* ❤

*Notebook version:* **0.28**

---


This notebook is designed to be used with training Retreival-Based Voice Conversion models without using a WеbUl. This is done to stay within the scope of Colab's TOS (and also because I personally don't like WеbUls.)

This notebook is *not* designed to be used for inference. Colab should not be used for RVC inference in general, use local inference instead. (RVC can inference just fine on any dime-a-dozen CPU from the past 7 years, you seriously don't need Colab.)

⚠ WARNING: Unlike the original RVC notebook, *this notebook ***does not*** have autosave functionality* due to the massive amount of underlying stress that RVC's autosave features places on Colab to Drive communication. Please be careful and try to keep training sessions short.

In [ ]:
# You will likely want a disconnect prevention method to keep Colab from timing you out for "inactivity" during training.
# I won't be providing one here, but they're not hard to come by. Search around the Colab AI community and you'll find one. ;)

# Dependencies
Run me first!

In [ ]:
#@title Install Dependencies
#Credit to Nick088 for fixing this cell on short order from the Fairseq bugginess.
import subprocess

packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
pip_packages = ['setuptools', 'wheel', 'faiss-gpu', 'fairseq', 'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5', 'numba==0.56.4', 'librosa==0.9.2']

print("Updating and installing system packages...")
for package in packages:
  print(f"Installing {package}...")
  subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

print("Updating and installing pip packages...")
# Install pip packages individually to identify problematic ones

# downgrade pip
!pip install --upgrade pip==23.1
# install packages
for package in pip_packages:
  print(f"Installing {package}...")
  subprocess.check_call(['pip', 'install', '--upgrade', package])

print('Packages up to date.')
firsttry = True

Updating and installing system packages...
Installing build-essential...
Installing python3-dev...
Installing ffmpeg...
Installing aria2...
Updating and installing pip packages...
Installing setuptools...
Installing wheel...
Installing faiss-gpu...
Installing fairseq...
Installing ffmpeg...
Installing ffmpeg-python...
Installing praat-parselmouth...
Installing pyworld...
Installing numpy==1.23.5...
Installing numba==0.56.4...
Installing librosa==0.9.2...
Packages up to date.


In [ ]:
#@title Clone Repositories
import os

# READ ME BEFORE CHANGING THINGS
# If you're attempting to replace the imports here with Applio-RVC, it will not work due to requirement discrepancies across the entire notebook.
# I will not be porting this notebook to Applio due to the failure of the Applio team to provide backwards compatibility with the Crepe and Mangio-Crepe f0 feature format.
# DO NOT ASK. IT WILL NOT HAPPEN.

os.chdir('/content/')

if(os.path.exists("/content/Mangio-RVC-Fork")):
  print("RVC already installed, skipping.")
else:
  #Credit to miaaaa0a on the AI Hub Discord for (indirectly) suggesting this variant of Mangio RVC to me.
  !git clone -b pr-optimization --single-branch https://github.com/alexlnkp/Mangio-RVC-Tweaks.git
  #Rename to keep backwards compatibility with old variants of Disconnected
  os.rename("/content/Mangio-RVC-Tweaks", "/content/Mangio-RVC-Fork")
  !git clone https://github.com/maxrmorrison/torchcrepe.git
  !mv torchcrepe/torchcrepe Mangio-RVC-Fork/
  !rm -rf torchcrepe  # Delete the torchcrepe repository folder

os.chdir('/content/Mangio-RVC-Fork')
now_dir = "/content/Mangio-RVC-Fork"
os.makedirs(os.path.join(now_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(now_dir, "weights"), exist_ok=True)

RVC already installed, skipping.


In [ ]:
#@title GPU Check
import torch

ngpu = torch.cuda.device_count()
gpu_infos = []
mem = []
if_gpu_ok = False

if torch.cuda.is_available() or ngpu != 0:
  for i in range(ngpu):
    gpu_name = torch.cuda.get_device_name(i)
    if any(
        value in gpu_name.upper()
        for value in ["10", "16", "20", "30", "40", "A2", "A3", "A4", "P4", "A50", "500", "A60", "70", "80", "90", "M4", "T4", "TITAN"]
    ):
      if_gpu_ok = True
      print("Compatible GPU detected: %s" % gpu_name)
      gpu_infos.append("%s\t%s" % (i, gpu_name))
      mem.append(int(torch.cuda.get_device_properties(i).total_memory / 1024 / 1024 / 1024 + 0.4))

if if_gpu_ok and len(gpu_infos) > 0:
  gpu_info = "\n".join(gpu_infos)

else:
  raise Exception("No GPU detected; training cannot continue. Please change your runtime type to a GPU.")
gpus = "-".join(i[0] for i in gpu_infos)

Compatible GPU detected: Tesla T4


In [ ]:
#@title Mount Drive
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print('Drive is already mounted. Proceed.')

os.makedirs('/content/drive/MyDrive/rvcDisconnected', exist_ok=True)

Drive is already mounted. Proceed.


In [ ]:
#@title Download Extra Files
#Didn't ask.

#Hubert/RMVPE
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/hubert_base.pt -d /content/Mangio-RVC-Fork -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/rmvpe.pt -d /content/Mangio-RVC-Fork -o rmvpe.pt

#FM JSONs
!rm -rf /content/Mangio-RVC-Fork/configs/32k.json
!rm -rf /content/Mangio-RVC-Fork/configs/40k.json
!rm -rf /content/Mangio-RVC-Fork/configs/48k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/32k.json -d /content/Mangio-RVC-Fork/configs -o 32k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/40k.json -d /content/Mangio-RVC-Fork/configs -o 40k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/48k.json -d /content/Mangio-RVC-Fork/configs -o 48k.json


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
542c15|OK  |       0B/s|/content/Mangio-RVC-Fork/hubert_base.pt

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
f3e6c8|OK  |       0B/s|/content/Mangio-RVC-Fork/rmvpe.pt

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
cd4eda|OK  |   127KiB/s|/content/Mangio-RVC-Fork/configs/32k.json

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
43a33c|OK  |   145KiB/s|/content/Mangio-RVC-Fork/configs/40k.json

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
==

In [ ]:
#@title Setup CSVDB
#...Alright, you made your point.
import csv

if not os.path.isdir("csvdb/"):
  os.makedirs("csvdb")
  frmnt, stp = open("csvdb/formanting.csv", "w", newline=""), open("csvdb/stop.csv", "w", newline="")
  csv_writer = csv.writer(frmnt, delimiter=",")
  csv_writer.writerow([False, 1.0, 1.0])
  csv_writer = csv.writer(stp, delimiter=",")
  csv_writer.writerow([False])
  frmnt.close()
  stp.close()

global DoFormant, Quefrency, Timbre
DoFormant, Quefrency, Timbre = False, 1.0, 1.0

# Initial Setup
You need to run these cells for every training session.

In [ ]:
#@title Set Training Variables
now_dir = "/content/Mangio-RVC-Fork"
experiment_name = "Nethaji1" #@param {type:"string"}
path_to_training_folder = "/content/dataset/"
model_architecture = "v2" #@param ["v1","v2"] {allow-input: false}
target_sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}
speaker_id = 0 #@param {type:"integer"}
pitch_extraction_algorithm = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
crepe_hop_length = 64 #@param {type:"integer"}
pitch_guidance = True #@param {type:"boolean"}

cpu_threads = !nproc
cpu_threads = int(cpu_threads[0])

exp_dir = f"{now_dir}/logs/{experiment_name}"

assert crepe_hop_length!=None, "You need to input something for crepe_hop_length, silly."
assert crepe_hop_length>0, "Hop length must be more than 0."
assert crepe_hop_length<=512, "Save frequency must be less than 512."

if(experiment_name == "experiment_name"):
  print("Warning: Your experiment name should be changed to the name of your dataset.")

In [ ]:
#@title Download Pretrained Model
#@markdown What pretrained model would you like to use?
pretrain_type = "OV2Super" #@param ["original", "OV2Super", "RIN_E3", "ItaIla", "SnowieV3", "SnowieV3xRIN_E3", "TITAN", "custom"] {allow-input: false}
#@markdown **Advanced:** Set your pretrain type to "custom" and input HuggingFace links to your G and D files here.
g="" #@param {type:"string"}
d="" #@param {type:"string"}
assert 'model_architecture' in locals(), "Hold up! You need to run Set Training Variables first."

if pretrain_type!="original" and model_architecture!="v2":
  model_architecture="v2"
  print("The new pretrains only support RVC v2 at this time. Your settings have been automatically adjusted.")

if pretrain_type=="OV2Super" and target_sample_rate=="48k":
  target_sample_rate="40k"
  print("OV2Super only supports 40k and lower at this time. Your settings have been automatically adjusted.")
if pretrain_type=="RIN_E3" and target_sample_rate!="40k":
  target_sample_rate="40k"
  print("RIN_E3 only supports 40k at this time. Your settings have been automatically adjusted.")
if pretrain_type=="ItaIla" and target_sample_rate!="32k":
  target_sample_rate="32k"
  print("ItaIla only supports 32k at this time. Your settings have been automatically adjusted.")
if pretrain_type=="SnowieV3xRIN_E3" and target_sample_rate!="40k":
  target_sample_rate="40k"
  print("SnowieV3 x RIN_E3 only supports 40k at this time. Your settings have been automatically adjusted.")

#Setup link
print("Setting up...")
if pretrain_type=="original" and model_architecture=="v2":
  g = f"https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G{target_sample_rate}.pth"
  d = f"https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D{target_sample_rate}.pth"
if pretrain_type=="original" and model_architecture=="v1":
  g = f"https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G{target_sample_rate}.pth"
  d = f"https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D{target_sample_rate}.pth"
if pretrain_type=="OV2Super" and target_sample_rate=="40k":
  g = f"https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super{target_sample_rate}G.pth"
  d = f"https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super{target_sample_rate}D.pth"
if pretrain_type=="OV2Super" and target_sample_rate=="32k":
  g = f"https://huggingface.co/poiqazwsx/Ov2Super32kfix/resolve/main/f0Ov2Super32kD.pth"
  d = f"https://huggingface.co/poiqazwsx/Ov2Super32kfix/resolve/main/f0Ov2Super32kD.pth"
if pretrain_type=="RIN_E3":
  g = "https://huggingface.co/MUSTAR/RIN_E3/resolve/main/RIN_E3_G.pth"
  d = "https://huggingface.co/MUSTAR/RIN_E3/resolve/main/RIN_E3_D.pth"
if pretrain_type=="SnowieV3":
  g = f"https://huggingface.co/MUSTAR/SnowieV3.1-{target_sample_rate}/resolve/main/G_SnowieV3.1_{target_sample_rate}.pth"
  d = f"https://huggingface.co/MUSTAR/SnowieV3.1-{target_sample_rate}/resolve/main/D_SnowieV3.1_{target_sample_rate}.pth"
if pretrain_type=="ItaIla":
  g = "https://huggingface.co/TheStinger/itaila/resolve/main/ItaIla_32k_G.pth"
  d = "https://huggingface.co/TheStinger/itaila/resolve/main/ItaIla_32k_D.pth"
if pretrain_type=="SnowieV3xRIN_E3":
  g = "https://huggingface.co/MUSTAR/SnowieV3.1-X-RinE3-40K/resolve/main/G_Snowie-X-Rin_40k.pth"
  d = "https://huggingface.co/MUSTAR/SnowieV3.1-X-RinE3-40K/resolve/main/D_Snowie-X-Rin_40k.pth"
if pretrain_type=="TITAN":
  g = f"https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/{target_sample_rate}/pretrained/G-f0{target_sample_rate}-TITAN-Medium.pth"
  d = f"https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/{target_sample_rate}/pretrained/D-f0{target_sample_rate}-TITAN-Medium.pth"

pretrained_base = "pretrained/" if model_architecture == "v1" else "pretrained_v2/"
unpt = f"_{pretrain_type}" if pretrain_type!="original" else ""

print("Downloading your pretrained model...")
!aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M {g} -d /content/Mangio-RVC-Fork/{pretrained_base} -o f0G{target_sample_rate}{unpt}.pth
!aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M {d} -d /content/Mangio-RVC-Fork/{pretrained_base} -o f0D{target_sample_rate}{unpt}.pth

print("Pretrain downloaded. Best of luck training!")

Setting up...
Pretrain downloaded. Best of luck training!


#Preprocessing
You should only need to run these cells once per model.

In [ ]:
#@title Load Dataset
#@markdown If it doesn't already exist, create a folder in your Google Drive named 'rvcDisconnected' and place your zip file there. This will look for the following ZIP file inside that 'rvcDisconnected' folder.
dataset = "Nethaji_input_audio.zip"  #@param {type:"string"}

#@markdown This loader will load datasets in a similar fashion to the So-Vits-SVC dataset loader. For best results, your dataset should be formatted as such:
#@markdown ```
#@markdown zipfile.zip
#@markdown └───character_name
#@markdown     ├───file1.wav
#@markdown     ├───...
#@markdown     └───file999.wav
#@markdown ```
#@markdown Audio filenames do not matter. All audio files should be in WAV format for best compatibility.

# TODO: Add something to convert non-WAVs to WAV

import os
import shutil

directories=[]

def sanitize_directory(directory):
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):
      if filename == ".DS_Store" or filename.startswith("._") or not filename.endswith(('.wav', '.flac', '.mp3', '.ogg', '.m4a')):
        os.remove(file_path)
    elif os.path.isdir(file_path):
      #Get rid of the MACOSX directory just so it doesn't mess with renaming later
      if(filename == "__MACOSX"):
        shutil.rmtree(file_path)
        continue
      #Append the directory to directories for future dataset check, then recurse.
      directories.append(file_path)
      sanitize_directory(file_path)

dataset_path = '/content/drive/MyDrive/rvcDisconnected/' + dataset
final_directory = '/content/dataset'
temp_directory = '/content/temp_dataset'

if os.path.exists(final_directory):
  print("Dataset folder already found. Wiping...")
  shutil.rmtree(final_directory)
if os.path.exists(temp_directory):
  print("Temporary folder already found. Wiping...")
  shutil.rmtree(temp_directory)

if not os.path.exists(dataset_path):
  raise Exception(f'I can\'t find {dataset} in {os.path.dirname(dataset_path)}.')

os.makedirs(final_directory, exist_ok=True)
os.makedirs(temp_directory, exist_ok=True)
#Oops.
!unzip -d "{temp_directory}" -B "{dataset_path}"
print("Sanitizing...")
sanitize_directory(temp_directory)

if(len(directories) == 0):
  #If there's no directories, we're dealing with a ZIP of just audio files.
  #Move everything to /dataset/experiment_name/.
  print("Dataset Type: Audio Files (Single Speaker)")
  expDir=os.path.join(final_directory, experiment_name)
  os.makedirs(expDir, exist_ok=True)
  for r, _, f in os.walk(temp_directory):
    for name in f:
      !cp "{temp_directory}/{name}" "{expDir}"
elif(len(directories) == 1):
  #If there's only one directory, we're dealing with a single speaker.
  #Rename the folder to experiment_name and move it to /dataset/.
  print("Dataset Type: Single Speaker")
  fi = os.path.join(temp_directory, experiment_name)
  os.rename(directories[0], fi)
  shutil.move(fi, final_directory)

else:
  #If anything else, we're dealing with multispeaker.
  #Move all folders to /dataset/ indiscriminately.
  print("Dataset Type: Multispeaker")
  for fi in directories:
    shutil.move(fi, final_directory)

shutil.rmtree(temp_directory)

print("Dataset imported.")


Dataset folder already found. Wiping...
Archive:  /content/drive/MyDrive/rvcDisconnected/Nethaji_input_audio.zip
  inflating: /content/temp_dataset/NethajiReddy/Sound01.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound02.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound03.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound04.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound05.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound06.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound07.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound08.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound09.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound10.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound11.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound12.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound13.wav  
  inflating: /content/temp_dataset/NethajiReddy/Sound14.wav  
  inflating: /conte

In [ ]:
#@title Preprocessing and Feature Extraction

import os
import subprocess

assert cpu_threads>0, "CPU threads not allocated correctly."

sr = int(target_sample_rate.rstrip('k'))*1000
pttf = path_to_training_folder + experiment_name
os.makedirs(f"{exp_dir}", exist_ok=True)

cmd = f"python trainset_preprocess_pipeline_print.py \"{pttf}\" {sr} {cpu_threads} \"{exp_dir}\" 1"
print(cmd)
!$cmd

gpuList = gpus.split("-")
cmd = f"python extract_f0_print.py \"{exp_dir}\" {cpu_threads} {pitch_extraction_algorithm} {crepe_hop_length}"
print(cmd)
!$cmd

leng = len(gpus)
cmd = f"python extract_feature_print.py \"device\" {leng} 0 0 \"{exp_dir}\" {model_architecture}"
print(cmd)
!$cmd

python trainset_preprocess_pipeline_print.py "/content/dataset/Nethaji1" 40000 2 "/content/Mangio-RVC-Fork/logs/Nethaji1" 1
start preprocess
['trainset_preprocess_pipeline_print.py', '/content/dataset/Nethaji1', '40000', '2', '/content/Mangio-RVC-Fork/logs/Nethaji1', '1']
thread:0:   0% 0/11 [00:00<?, ?it/s]
thread:1:   0% 0/10 [00:00<?, ?it/s]
thread:0:   9% 1/11 [00:05<00:59,  5.92s/it]
thread:0:  18% 2/11 [00:10<00:46,  5.11s/it]
thread:0:  27% 3/11 [00:13<00:32,  4.00s/it]
thread:0:  36% 4/11 [00:15<00:24,  3.44s/it]
thread:0:  45% 5/11 [00:18<00:18,  3.16s/it]
thread:0:  55% 6/11 [00:21<00:15,  3.15s/it]
thread:0:  64% 7/11 [00:25<00:14,  3.54s/it]
thread:0:  73% 8/11 [00:29<00:10,  3.48s/it]
thread:0:  82% 9/11 [00:31<00:06,  3.21s/it]
thread:1: 100% 10/10 [00:33<00:00,  3.38s/it]
thread:0: 100% 11/11 [00:35<00:00,  3.22s/it]
end preprocess
python extract_f0_print.py "/content/Mangio-RVC-Fork/logs/Nethaji1" 2 rmvpe 64
['extract_f0_print.py', '/content/Mangio-RVC-Fork/logs/Nethaji

In [ ]:
#@title Save preprocessed dataset files to Google Drive
#Compress dataset folder
loc = "%s/logs/%s" % (now_dir, experiment_name)
!zip -r rvcLogs.zip "{loc}"
DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
!mkdir -p "{DATASET_PATH_DRIVE}"
!cp /content/Mangio-RVC-Fork/rvcLogs.zip "{DATASET_PATH_DRIVE}"

updating: content/Mangio-RVC-Fork/logs/Nethaji1/ (stored 0%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/ (stored 0%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/11_25.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/20_0.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/13_3.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/6_3.wav (deflated 5%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/3_20.wav (deflated 5%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/17_17.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/5_0.wav (deflated 5%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/3_10.wav (deflated 5%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/18_22.wav (deflated 5%)
updating: content/Mangio-RVC-Fork/logs/Nethaji1/1_16k_wavs/11_11.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/Nethaj

# Training
Also includes resuming code.

In [ ]:
#@title Load preprocessed dataset files from Google Drive (for resuming)
#@markdown If you already have preprocessed dataset files on Google Drive, you can load them here instead of re-running the preprocessing steps.
import os

BACK_UP_DATASET_PATH = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name

#Prevent people from loading the ZIP over existing files
ok=True
if(os.path.exists("/content/Mangio-RVC-Fork/logs/"+experiment_name+"/2a_f0")):
  print("Dataset files already loaded, skipping.")
  ok=False

if ok:
  !unzip "{BACK_UP_DATASET_PATH}/rvcLogs.zip" -d /

In [ ]:
#@title Import Model from Drive to Notebook (for resuming)
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
#@markdown Input the model's Step Count here (the number located on your model's G and D files.) If you used `save_only_last_ckpt` during training, this number will be 2333333.
STEPCOUNT = 000 #@param {type:"integer"}

print("Copying model files...")
!cp "{DATASET_PATH_DRIVE}/D_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/G_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/config.json" "{DATASET_PATH_COLAB}"

print("Copying Tensorboard TFEVENT files...")
for r, _, f in os.walk(DATASET_PATH_DRIVE):
  for name in f:
    if(name.startswith("events.out.tfevents")):
      !cp "{DATASET_PATH_DRIVE}/{name}" "{DATASET_PATH_COLAB}"

print("All done. Welcome back!")

In [ ]:
#@title Index Training
#@markdown Ensure that Feature Extraction has run successfully before running this cell.

#@markdown Use this option if you wish to save the two extra files generated by index training to your Google Drive. (Only the added index is normally needed.)
save_extra_files_to_drive = False #@param {type:"boolean"}

#@markdown **Advanced:** Select this option if you would like to force the trainer to use Minibatch K-Means instead of Faiss. *Do not enable this unless you know what you're doing.*
force_mbkm = False #@param {type:"boolean"}

#Oh dear lord why is this baked into infer-web I hate this
import os
import sys
import traceback
import numpy as np
import faiss

#from sklearn.cluster import MiniBatchKMeans

exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
os.makedirs(exp_dir, exist_ok=True)
feature_dir = (
    "%s/3_feature256" % (exp_dir)
    if model_architecture == "v1"
    else "%s/3_feature768" % (exp_dir)
)
print(feature_dir)
if not os.path.exists(feature_dir):
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")
listdir_res = list(os.listdir(feature_dir))
if len(listdir_res) == 0:
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")

try:
  from sklearn.cluster import MiniBatchKMeans
except:
  print("Due to a bug with Colab, we will need to reinstall Numpy real quick. Give me a sec!")
  !pip install -U numpy
  print("Numpy reinstalled. Please restart the runtime, and then re-run the \"Set Training Variables\" cell to continue.")
  sys.exit()
else:
  print("Proper Numpy version detected.")

infos=[]
npys=[]
for name in sorted(listdir_res):
  phone = np.load("%s/%s" % (feature_dir, name))
  npys.append(phone)
big_npy = np.concatenate(npys, 0)
big_npy_idx = np.arange(big_npy.shape[0])
np.random.shuffle(big_npy_idx)
if big_npy.shape[0] > 2e5 or force_mbkm:
  print("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
  try:
    big_npy = (
        MiniBatchKMeans(
            n_clusters=10000,
            verbose=True,
            batch_size=256,
            compute_labels = False,
            init="random"
        )
        .fit(big_npy)
        .cluster_centers_

    )
  except:
    info = traceback.format_exc()
    print(info)

np.save("%s/total_fea.npy" % exp_dir, big_npy)
n_ivf = min(int(16*np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
print("%s,%s" % (big_npy.shape, n_ivf))
index = faiss.index_factory(256 if model_architecture == "v1" else 768, "IVF%s,Flat" % n_ivf)
print("Training index...")
index_ivf = faiss.extract_index_ivf(index)
index_ivf.nprobe = 1
index.train(big_npy)
faiss.write_index(
    index,
    "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index" % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)
print("Adding...")
batch_size_add = 8192
for i in range(0, big_npy.shape[0], batch_size_add):
  index.add(big_npy[i:i+batch_size_add])
faiss.write_index(
    index,
    "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
    % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)

npr = index_ivf.nprobe

print("Saving files to Drive...")
DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p "{DATASET_PATH_DRIVE}"
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(save_extra_files_to_drive):
  !cp "{DATASET_PATH_COLAB}/total_fea.npy" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/trained_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index" "{DATASET_PATH_DRIVE}"
!cp "{DATASET_PATH_COLAB}/added_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index" "{DATASET_PATH_DRIVE}"

print("All done! Your index file has completed training.")
try:
  firsttry
except:
  print("If you had to restart the runtime, disconnect and delete the runtime in order to continue. (Restarting the runtime again will not work.)")



/content/Mangio-RVC-Fork/logs/Nethaji1/3_feature768
Proper Numpy version detected.
(54655, 768),1401
Training index...
Adding...
Saving files to Drive...
All done! Your index file has completed training.


In [ ]:
import os
import math
from random import shuffle


assert 'model_architecture' in locals(), "Hold up! You need to run the  \"Set Training Variables\" cell first."


assert 'pretrain_type' in locals(), "You need to download a pretrain! Please run the \"Download Pretrained Model\" cell before continuing."


#@title Training
save_frequency = 50 #@param {type:"integer"}
total_epochs = 1024 #@param {type:"integer"}
batch_size = 8 #@param {type:"integer"}
save_only_latest_ckpt = True #@param {type:"boolean"}
cache_all_training_sets = False #@param {type:"boolean"}
save_small_final_model = True #@param {type:"boolean"}
#@markdown **Advanced:** You can manually define a log interval here.
use_manual_stepToEpoch = False #@param {type:"boolean"}
manual_stepToEpoch = 000 #@param {type:"integer"}

assert save_frequency!=None, "You need to input something for save_frequency, silly."
assert save_frequency>0, "Save frequency must be more than 0."
if(save_frequency>50):print(f"...A save frequency of {save_frequency}? A bit high, but... alright then.")
assert total_epochs!=None, "You need to input something for total_epochs, silly."
assert total_epochs>0, "Total epochs must be more than 0."
if(total_epochs>10000):print(f"...A total epoch count of of {total_epochs}? This is going to overtrain, but... alright then.")
assert batch_size!=None, "You need to input something for batch_size, silly."
assert batch_size>0, "Batch size must be more than 0."
assert batch_size<=40, "Batch size must be less than 40. (I'd reccomend a value between 6 and 12 for Colab.)"

pretrained_base = "pretrained/" if model_architecture == "v1" else "pretrained_v2/"
unpt = f"_{pretrain_type}" if pretrain_type!="original" else ""

pretrainedD = f"{pretrained_base}f0D{target_sample_rate}{unpt}.pth"
pretrainedG = f"{pretrained_base}f0G{target_sample_rate}{unpt}.pth"

#Log interval
log_interval = 1
liFolderPath = os.path.join(exp_dir, "1_16k_wavs")
if(os.path.exists(liFolderPath) and os.path.isdir(liFolderPath)):
  wav_files = [f for f in os.listdir(liFolderPath) if f.endswith(".wav")]
  if wav_files:
    sample_size = len(wav_files)
    log_interval = math.ceil(sample_size / batch_size)
    if log_interval > 1:
      log_interval += 1

if log_interval > 250 and not use_manual_stepToEpoch:
  print(f"That's a big dataset you got there. Log interval normalized to 200 steps from {log_interval} steps.")
  log_interval = 200

if use_manual_stepToEpoch:
  log_interval = manual_stepToEpoch

#Create Python command
cmd = "python train_nsf_sim_cache_sid_load_pretrain.py -e \"%s\" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s -li %s" % (
    experiment_name,
    target_sample_rate,
    1,
    batch_size,
    0,
    total_epochs,
    save_frequency,
    "-pg %s" % pretrainedG if pretrainedG != "" else "\b",
    "-pd %s" % pretrainedD if pretrainedD != "" else "\b",
    1 if save_only_latest_ckpt else 0,
    1 if cache_all_training_sets else 0,
    1 if save_small_final_model else 0,
    model_architecture,
    log_interval,
)
print(cmd)

#Create mute filelist
gt_wavs_dir = f"{exp_dir}/0_gt_wavs"
feature_dir = (
  f"{exp_dir}/3_feature256"
  if model_architecture == "v1"
  else f"{exp_dir}/3_feature768"
)
f0_dir = f"{exp_dir}/2a_f0"
f0nsf_dir = f"{exp_dir}/2b-f0nsf"
names = (
  set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
  & set([name.split(".")[0] for name in os.listdir(feature_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
)
opt = []
for name in names:
  opt.append(
    "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
    % (
      gt_wavs_dir.replace("\\", "\\\\"),
      name,
      feature_dir.replace("\\", "\\\\"),
      name,
      f0_dir.replace("\\", "\\\\"),
      name,
      f0nsf_dir.replace("\\", "\\\\"),
      name,
      speaker_id,
    )
  )
fea_dim = 256 if model_architecture == "v1" else 768
for _ in range(2):
  opt.append(
      f"{now_dir}/logs/mute/0_gt_wavs/mute{target_sample_rate}.wav|{now_dir}/logs/mute/3_feature{fea_dim}/mute.npy|{now_dir}/logs/mute/2a_f0/mute.wav.npy|{now_dir}/logs/mute/2b-f0nsf/mute.wav.npy|{speaker_id}"
  )
shuffle(opt)
with open(f"{exp_dir}/filelist.txt", "w") as f:
  f.write("\n".join(opt))
print("Mute filelist written. Best of luck training!")


%cd /content/Mangio-RVC-Fork
%load_ext tensorboard
%tensorboard --logdir /content/Mangio-RVC-Fork/logs

os.chdir('/content/Mangio-RVC-Fork')
!$cmd


AssertionError: Hold up! You need to run the  "Set Training Variables" cell first.

In [ ]:
#@title Export Model from Notebook to Drive
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p "{DATASET_PATH_DRIVE}"

#@markdown **Advanced:** Use this option if you wish to only copy over weights.
skip_models = False #@param {type:"boolean"}
#@markdown **Advanced:** Use these options if you wish to manually input your step count and epoch count for incomplete models. *Do not use this option if your training finished.*
manual_save = False #@param {type:"boolean"}
STEPCOUNT = 000 #@param {type:"integer"}
EPOCHCOUNT = 000 #@param {type:"integer"}

finished=False
potential="/content/Mangio-RVC-Fork/weights/"+experiment_name+".pth"
if os.path.exists(potential):
  finished = True

#VERY hacky. Might break stuff, report to me if it does.
print("Detecting latest model...")
if(not manual_save):
  currentMax = 0
  for r, _, f in os.walk("/content/Mangio-RVC-Fork/weights/"):
    for name in f:
      if(name.endswith(".pth") and (name!=experiment_name+".pth")):
        #Check to see if this PTH is what we're looking for.
        if(name.find(experiment_name)==-1):
          continue
        #Determine Epochcount+Stepcount Phase 1
        pot = name.split('_')
        ep=pot[len(pot)-2][1:]
        #If what we got from the epoch count section of the filename isn't a number, multiple completed models are in weights.
        #Skip it if that happens.
        if(not ep.isdecimal()):
          continue
        #Determine Epochcount+Stepcount Phase 2
        ep=int(ep)
        if ep>currentMax:
          currentMax=ep
          step=pot[len(pot)-1].split('.')
          step=int(step[0][1:])
          EPOCHCOUNT=ep
          STEPCOUNT=step

TSTEP = STEPCOUNT
if(not skip_models):
  print("Copying model files...")
  if(save_only_latest_ckpt):
    TSTEP=2333333
  !cp "{DATASET_PATH_COLAB}/D_{TSTEP}.pth" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/G_{TSTEP}.pth" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/config.json" "{DATASET_PATH_DRIVE}"

print("Copying Tensorboard TFEVENT files...")
for r, d, f in os.walk(DATASET_PATH_COLAB):
  for name in f:
    if(name.startswith("events.out.tfevents") and os.path.exists(os.path.join(DATASET_PATH_COLAB, name))):
      !cp "{DATASET_PATH_COLAB}/{name}" "{DATASET_PATH_DRIVE}"

print("Copying weight file...")
if(finished):
  !cp "{potential}" "{DATASET_PATH_DRIVE}"
else:
  !cp "/content/Mangio-RVC-Fork/weights/{experiment_name}_e{EPOCHCOUNT}_s{STEPCOUNT}.pth" "{DATASET_PATH_DRIVE}"

print("All done!")

In [ ]:
#@title Export Finished Model to HuggingFace
#Credit to LollenApe for this cell!

#TODO:
#maybe remove glob dependency?

import os
import zipfile
import glob

assert experiment_name, "No experiment name found."

#@markdown **Advanced:** Use this option if you wish to manually use a specific epoch value. *Leave this blank if your model finished training.*
manual_epoch_number = "" #@param {type:"string"}
num_epochs = int(manual_epoch_number) if manual_epoch_number.isdigit() else None

# Construct the weights path based on the provided number of epochs
if num_epochs is not None:
  weights_path = f"/content/Mangio-RVC-Fork/weights/{experiment_name}_e{num_epochs}*"
else:
  print("Autodetecting epoch count...")
  potential = f"/content/Mangio-RVC-Fork/weights/{experiment_name}.pth"
  if os.path.exists(potential):
    weights_path = f"/content/Mangio-RVC-Fork/weights/{experiment_name}"
  else:
    currentMax = 0
    for r, _, f in os.walk("/content/Mangio-RVC-Fork/weights/"):
      for name in f:
        if(name.endswith(".pth") and (name!=experiment_name+".pth")):
          if(name.find(experiment_name)==-1):
            continue
          pot = name.split('_')
          ep=pot[len(pot)-2][1:]
          if(not ep.isdecimal()):
            continue
          ep=int(ep)
          if ep>currentMax:
            currentMax=ep
    num_epochs=currentMax
    weights_path = f"/content/Mangio-RVC-Fork/weights/{experiment_name}_e{num_epochs}*"
  print(f"Model with path {weights_path} found automatically.")

weights_files = glob.glob(weights_path + ".pth")
logs_path = f"/content/Mangio-RVC-Fork/logs/{experiment_name}/added_*_1_{experiment_name}_{model_architecture}.index"

print(f"Searching for weights files in: {weights_path}.pth")
print(f"Searching for logs file in: {logs_path}")

if weights_files and any(glob_result := glob.glob(logs_path)):
    log_file = glob_result[0]
    output_folder = "/content/toHF"
    os.makedirs(output_folder, exist_ok=True)
    output_zip_path = f"{output_folder}/{experiment_name}.zip"
    with zipfile.ZipFile(output_zip_path, 'w') as zipf:
        for weights_file in weights_files:
            zipf.write(weights_file, os.path.basename(weights_file))
        zipf.write(log_file, os.path.basename(log_file))
    print(f"The files have been added to the zip folder: {output_zip_path}")
else:
    print(f"Found weights files: {weights_files}")
    print(f"Found logs files: {glob_result}")
    raise Exception("Couldn't find your model files. Check the found file results above. (Did you run Index Training?)")

from huggingface_hub import login, HfApi
#@markdown Enter HuggingFace Token (set Role to 'write'):
hftoken = "Obtain this at https://huggingface.co/settings/tokens" #@param {type:"string"}

# Login
login(token=f"{hftoken}")

#@markdown Set this option to true if you wish to create a new repository for this model.
create_new_repo = False #@param {type:"boolean"}
#@markdown Enter the HuggingFace repository name you wish to send the model to:
repoid = "username/RepoName" #@param {type:"string"}

if create_new_repo:
    from huggingface_hub import create_repo
    create_repo(repoid)
    print(f"Repo '{repoid}' created successfully!")

api = HfApi()
# Upload folders
api.upload_folder(folder_path="/content/toHF",
                 repo_id=f"{repoid}",
                 repo_type="model")

# Changelogs, Credits, and Special Thanks

**Changelogs**

**v0.28** - Added Blaise-TK's TITAN pretrains.

Older changelogs can be found at https://rentry.co/rvcdisconnected_changelogs.

**Credits**
*   [Kit Lemonfoot](https://huggingface.co/Kit-Lemonfoot) - writing this notebook
*   [RVC Project](https://github.com/RVC-Project) - Created RVC, obviously
*   [Mangio261](https://github.com/Mangio621/) - Creating the Mangio RVC fork
*   [Kalomaze](https://github.com/kalomaze) - Original RVC colab + Mangio tweaks
*   [Alexlnkp](https://github.com/alexlnkp) - Created a more up-to-date variant of the Mangio-RVC fork
*   [LollenApe](https://huggingface.co/lollenape) - Created the "Export Finished Model to HuggingFace" cell
*   [Sonphantrung](https://github.com/sonphantrung) - Large amounts of code auditing and adjustments
*   [LJ1995](https://huggingface.co/lj1995) - Pretrained RVC models
*   [SimplCup](https://huggingface.co/SimplCup) - OV2Super pretrained RVC models
*   [MUSTAR](https://huggingface.co/MUSTAR) - RIN_E3 and SnowieRU pretrained RVC models
*   [TheStinger / Ilaria](https://huggingface.co/TheStinger) - ItaIla pretrained RVC models
*   [Blaise-TK](https://huggingface.co/blaise-tk) - TITAN pretrained RVC models
*   [Pony Preservation Project](https://boards.4channel.org/mlp/catalog#s=Pony%20Preservation%20Project) - for their robust TalkNet and So-Vits-SVC notebooks
*   the Colab team - forcing my hand and making me release this notebook early

**Special Thanks**
*   [Fifteen AI](https://15.ai/) - for getting me into voice AI in the first place
*   [Dacoolkid44](https://huggingface.co/dacoolkid44) / [HoloAI44](https://www.youtube.com/@Holo_AI44) and Hijack / [SANSSWEEP](https://huggingface.co/SANSSWEEP) - for basically kickstarting the larger VTuber voice AI scene with their models
*   [Maki Ligon](https://www.youtube.com/@Shiina_Mashiro) / [Yuuto Ichika](https://www.youtube.com/@yuutoichika) - for keeping me grounded in reality while developing this thing
*   [Bartezes](https://www.youtube.com/@bartezes3082) - for helping me so much with the [VTuber AI Model Tracking spreadsheet](https://docs.google.com/spreadsheets/d/1tvZSggOsZGAPjbMrWOAAaoJJFpJuQlwUEQCf5x1ssO8/)
*   [Megaaziib](https://www.youtube.com/@megaaziib) - for inspiring me to keep working on AI models and covers (I don't hate you)
*   [Saintlysaint](https://www.youtube.com/@SaintlySaint) and [Gengar2525](https://www.youtube.com/@GeGaCh) - for having faith in me
